# Task-4: Spam Detection using scikit-learn
Objective: Build a classifier (spam vs ham) using scikit-learn, evaluate it, and save the trained pipeline.

In [ ]:
# !pip install -q pandas scikit-learn matplotlib nltk joblib

import os, pandas as pd, numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
url = 'https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv'
df = pd.read_csv(url, sep='\t', header=None, names=['label','text'])

df['label'] = df['label'].str.lower().str.strip()
df['text'] = df['text'].astype(str)
df['label_num'] = df['label'].map({'ham':0, 'spam':1})
print("Dataset shape:", df.shape)
df.head()

In [ ]:
print(df['label'].value_counts())
df['text_len'] = df['text'].str.split().apply(len)
plt.hist(df['text_len'], bins=30)
plt.xlabel('Words per message'); plt.ylabel('Count'); plt.title('Message length distribution')
plt.show()

In [ ]:
X = df['text']
y = df['label_num']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Train/test sizes:", X_train.shape, X_test.shape)

In [ ]:
pipe = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', LogisticRegression(solver='liblinear', max_iter=500))
])

param_grid = {
    'tfidf__ngram_range': [(1,1),(1,2)],
    'tfidf__min_df': [1,2],
    'clf__C': [0.1, 1]
}

grid = GridSearchCV(pipe, param_grid, cv=3, scoring='f1', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)
best_model = grid.best_estimator_

In [ ]:
y_pred = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("\nClassification report:\n", classification_report(y_test, y_pred, target_names=['ham','spam']))

cm = confusion_matrix(y_test, y_pred, labels=[0,1])
print("Confusion matrix:\n", cm)

In [ ]:
model_path = 'spam_classifier_pipeline.joblib'
joblib.dump(best_model, model_path)
print("Saved model to", model_path)

In [ ]:
loaded = joblib.load(model_path)
examples = ["Congratulations! You won a prize.", "Hey, are you free today?"]
preds = loaded.predict(examples)
for t,p in zip(examples,preds):
    print(p, "-", t)

## Conclusion
- Logistic Regression with TF-IDF performed well on spam classification.
- Improvements could include more preprocessing, handling class imbalance, or trying deep learning models.
- Deliverables: notebook, saved model, requirements.txt, README.md.